In [1]:
# import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import xgboost
import pandas as pd
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [2]:
import numpy as np
import pandas as pd 
from tqdm import tqdm
import math
import time
import lightgbm as lgb
import seaborn as sns
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

tqdm.pandas()

In [3]:
train = pd.read_csv('/kaggle/input/cascade-cup-22/train.csv')
test = pd.read_csv('/kaggle/input/cascade-cup-22/test.csv')
sample_submission = pd.read_csv('/kaggle/input/cascade-cup-22/sample_submission.csv')

In [4]:
id_columns = ['order_id']

time_columns = ['order_time', 'order_date', 'allot_time', 'accept_time',
       'pickup_time', 'delivered_time','cancelled_time']

categorical_columns = ['reassignment_method','reassignment_reason']

# Converting to datetime and object


In [5]:
ids = train[id_columns]
train = train.drop(ids, axis=1)
test = test.drop(ids,axis=1)

for col in time_columns:
    train[col] = pd.to_datetime(train[col])
    try:
        test[col] = pd.to_datetime(test[col])
    except:
        continue

for col in categorical_columns:
    train[col] = train[col].astype('object')
    try:
        test[col] = test[col].astype('object')
    except:
        continue


# Preprocessing

In [6]:
def peak_hour(x):
    if x>=8 and x<=11:
        return 1
    if x>=13 and x<=19:
        return 1
    return 0

def log_normal(x):
    if x==0:
        return 0
    if x>0:
        return math.log(x)
    else:
        return math.log(abs(x))

In [7]:
def preprocess(df):
    df['order_day'] = df['order_date'].dt.dayofyear
    df['diff_allot_order'] = df['allot_time']-df['order_time']
    df['diff_allot_order'] = df['diff_allot_order'].dt.components['hours']*3600+df['diff_allot_order'].dt.components['minutes']*60+df['diff_allot_order'].dt.components['seconds']
    df['diff_accept_allot'] = df['accept_time']-df['allot_time']
    df['diff_accept_allot'] = df['diff_accept_allot'].dt.components['hours']*3600+df['diff_accept_allot'].dt.components['minutes']*60+df['diff_accept_allot'].dt.components['seconds']
    df['diff_accept_order'] = df['accept_time']-df['order_time']
    df['diff_accept_order'] = df['diff_accept_order'].dt.components['hours']*3600+df['diff_accept_order'].dt.components['minutes']*60+df['diff_accept_order'].dt.components['seconds']
    df['total_dist'] = df['first_mile_distance']+df['last_mile_distance']
    df['hour_in_day'] = df['order_time'].dt.hour
    df['day_of_week'] = df['order_date'].dt.dayofweek
    df[['cum_diff_allot_order','cum_diff_accept_allot','cum_diff_accept_order','cum_total_dist','cum_first_mile_dist','cum_last_mile_dist']]=df.groupby(['rider_id','order_date'])[['diff_allot_order','diff_accept_allot','diff_accept_order','total_dist','first_mile_distance','last_mile_distance']].cumsum()
    df['order_today'] = df.groupby(['rider_id','order_date']).cumcount()
    df['orders_cum'] = df.groupby(['rider_id']).cumcount()
    df['reassigned_order'] = df['reassigned_order'].fillna(0)                                                      
    df['saturday'] = df['day_of_week'].apply(lambda x:1 if x==5 else 0)
    for col in df.columns:
        if df[col].isnull().sum()>0:
            df[col]=df[col].fillna(0)
    df['orders_diff'] = df['alloted_orders']-df['delivered_orders']
    riders = df.groupby(['order_date','hour_in_day'])['rider_id'].count().to_dict()
    df['riders_available'] = df.apply(lambda x: riders[(x.order_date,x.hour_in_day)],axis=1)
    df['reliability'] = (df['alloted_orders']-df['undelivered_orders'])*100/df['alloted_orders']
    df['reliability'] = df['reliability'].fillna(0)
    cols = []
    col_dict = ((df.dtypes==int)|(df.dtypes==float)).to_dict()
    for col in col_dict.keys():
        if col_dict[col]:
            cols.append(col)
    df["mean"] = df[cols].mean(axis=1)
    df["std"] = df[cols].std(axis=1)
    df["min"] = df[cols].min(axis=1)
    df["max"] = df[cols].max(axis=1) 
    
    return df

# Train Valid Split

In [8]:
train_df = train[:395778]
valid_df = train[395778:]
train_df = preprocess(train_df)
valid_data = preprocess(valid_df)
test_df= preprocess(test)

# Rider details feature engineering

In [9]:
rider_cols_mean = ['session_time', 'first_mile_distance','last_mile_distance','total_dist']

df = pd.concat([train_df,valid_df,test_df])

data = df.groupby('rider_id').cumsum()[rider_cols_mean]
data['cumcounts'] = df.groupby('rider_id').cumcount()
data['cumcounts'] = data['cumcounts']+1
for i in ['session_time', 'first_mile_distance', 'last_mile_distance', 'total_dist']:
    data[i] = data[[i,'cumcounts']].progress_apply(lambda x:x[i]/x['cumcounts'],axis=1)

In [10]:
rider_deets = data.copy()
rider_deets

In [11]:
time_columns = ['order_time', 'order_date', 'allot_time', 'accept_time',
       'pickup_time', 'delivered_time','cancelled_time']
numerical_columns = ['first_mile_distance',
       'last_mile_distance', 'alloted_orders', 'delivered_orders', 'lifetime_order_count',
                    'session_time','diff_allot_order', 'diff_accept_allot',
       'diff_accept_order', 'total_dist','cum_diff_allot_order', 'cum_diff_accept_allot',
       'cum_diff_accept_order', 'cum_total_dist', 'allotted_orders',
                     'riders_available','orders_cum','reliability']
categorical_columns = ['reassignment_method','reassignment_reason','reassigned_order','order_day','hour_in_day','undelivered_orders','day_of_week',
                       'order_today', 'allotted', 'deliveries','period_in_day','saturday','first_mile', 'last_mile','orders_diff']

# Undersampling negative class and combining

In [12]:
train_df_c = train_df[train_df['cancelled']==1].sample(int(train_df[train_df['cancelled']==1].shape[0]*1),replace=False,random_state=42)
train_df_d = train_df[train_df['cancelled']==0].sample(int(train_df_c.shape[0])*1,random_state=42)

In [13]:
combined_df = pd.concat([train_df_c,train_df_d])
combined_df = combined_df.sort_values('order_time')

In [14]:
cols = ['first_mile_distance',
       'last_mile_distance', 'alloted_orders', 'delivered_orders',
       'undelivered_orders', 'lifetime_order_count',
       'session_time',
       'order_day', 'diff_allot_order', 'diff_accept_allot',
       'diff_accept_order', 'total_dist', 'hour_in_day', 'day_of_week','cum_diff_allot_order', 'cum_diff_accept_allot',
       'cum_diff_accept_order','reassigned_order',
       'cum_total_dist', 'order_today','rider_id','orders_cum',
       'saturday','orders_diff','riders_available','cum_first_mile_dist','cum_last_mile_dist','reliability','mean','std','min','max']

# Combining datasets with details about the rider

In [15]:
combined_df = combined_df[cols+['cancelled']]
combined_df = pd.merge(combined_df,rider_deets[:-test_df.shape[0]],left_index=True,right_index=True)

X_train = combined_df.drop('cancelled',axis=1)
y_train = combined_df['cancelled']


valid_df = valid_data[cols+['cancelled']]
valid_df = pd.merge(valid_df,rider_deets[:-test_df.shape[0]],left_index=True,right_index=True)

X_test = valid_df.drop('cancelled',axis=1)
y_test = valid_df['cancelled']

# Predictions using Random Forests Classifier

In [16]:
params = {'n_estimators': 900, 'max_depth': 40, 'random_state': 2020, 'min_samples_split': 10, 'min_samples_leaf': 2}

rf = RandomForestClassifier(**params)
rf.fit(X_train, y_train)

In [17]:
rf_preds = rf.predict(X_test)
rf_preds_prob = rf.predict_proba(X_test)[:,1]

In [18]:
result = confusion_matrix(y_test, rf_preds)
r = np.sum(result,axis=1)
a,b,c,d=result[0][0]/r[0],result[0][1]/r[0],result[1][0]/r[1],result[1][1]/r[1]
sns.heatmap([[a,b],[c,d]], annot=True, fmt=".2%", cmap='Blues')
print(classification_report(y_test,rf_preds))
print('Test Accuracy: ',rf.score(X_test,y_test))
print('Train Accuracy: ',rf.score(X_train,y_train))
print('AUC Score:',roc_auc_score(y_test,rf_preds_prob))

# XGBoost Classifier

In [19]:
params = {'lambda': 0.20177241158844367, 'alpha': 3.4670054832209787, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.02, 'max_depth': 5, 'random_state': 48, 'min_child_weight': 37}

xgb_model = xgb.XGBClassifier(**params)
xgb_model.fit(X_train, y_train,eval_metric='auc')
xgby_pred = xgb_model.predict(X_test)
xgby_preds_prob = xgb_model.predict_proba(X_test)[:,1]

In [20]:
result = confusion_matrix(y_test, xgby_pred)
r = np.sum(result,axis=1)
a,b,c,d=result[0][0]/r[0],result[0][1]/r[0],result[1][0]/r[1],result[1][1]/r[1]
sns.heatmap([[a,b],[c,d]], annot=True, fmt=".2%", cmap='Blues')
result1 = classification_report(y_test, xgby_pred)
print("Classification Report:",)
result2 = accuracy_score(y_test,xgby_pred)
print('Train Accuracy: ',xgb_model.score(X_train,y_train))
print("Test Accuracy:",result2)
print('AUC Score:',roc_auc_score(y_test,xgby_preds_prob))

In [21]:
check = pd.DataFrame()
check['rf_pred'] = rf_preds_prob
check['xgb_pred'] = xgby_preds_prob
check['sums'] = check[['rf_pred','xgb_pred']].mean(axis=1)
check['cancelled'] = y_test.reset_index()['cancelled']
check['pred'] = check['sums'].apply(lambda x: 1 if x>=0.5 else 0)
result = confusion_matrix(y_test, check['pred'])
print(result)
a,b,c,d=result[0][0]/r[0],result[0][1]/r[0],result[1][0]/r[1],result[1][1]/r[1]
sns.heatmap([[a,b],[c,d]], annot=True, fmt=".2%", cmap='Blues')
print(classification_report(y_test, check['pred']))
print("AUC Score: ",roc_auc_score(y_test,check['sums']))

# Predicting on test data and submission

In [22]:
test_df = test_df[cols]
test_data = pd.merge(test_df,rider_deets[rider_deets.shape[0]//2:],left_index=True,right_index=True)

In [23]:
y_preds_rf = rf.predict_proba(test_data)[:,1]
y_preds_xgb = xgb_model.predict_proba(test_data)[:,1]


In [24]:
sub = pd.DataFrame(columns = ['order_id'])
sub['order_id'] = pd.read_csv('../input/cascade-cup-22/test.csv')['order_id']
sub['rf_pred'] = y_preds_rf
sub['xgb_pred'] = y_preds_xgb
sub['sums'] = sub[['rf_pred','xgb_pred']].mean(axis=1)
sub['cancelled'] = sub['sums'].apply(lambda x: x)
sub

In [25]:
sns.kdeplot(sub['cancelled'])

In [26]:
sub[['order_id','cancelled']].to_csv('submission.csv',index=False)